In [1]:
# ==============================================================================
# GPR – Thesis Tuning Pipeline
# ==============================================================================
# Spezifikation gemäß Masterarbeit:
# - Setup I & II: Volles FE-Grid (inkl. DR, auch wenn GPR teuer ist).
# - Model: ARD-RBF, ARD-Matérn (nu=1.5, 2.5).
# - Optimization: n_restarts gemäß Thesis Auswahl.
# - Stage A Shortlist: Top 10 frozen.
# - Dynamic FI (Setup III): Strikt Top 50 Features.
#
# WARNUNG: GPR skaliert kubisch O(N^3). Das Grid hier ist mächtig.
# Ggf. auf kleineren Maschinen n_restarts reduzieren.
# ==============================================================================

import os, sys
from pathlib import Path
import numpy as np
import pandas as pd
from itertools import product

# --- 1) Pfad-Setup ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# --- 2) Imports ---
from src.config import (
    GlobalConfig,
    DEFAULT_CORR_SPEC,
    EWMA_CORR_SPEC,
    outputs_for_model,
)
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import (
    load_target,
    load_ifo_features,
    load_full_lagged_features,
    load_rolling_importance,
)
# WICHTIG: Modell-Import für GPR
from src.models.GPR import ForecastModel

# --- 3) Konfiguration ---
USE_DYNAMIC_FI_PIPELINE = False  # False = Standard Setup (I & II)
SEED = 42

if USE_DYNAMIC_FI_PIPELINE:
    MODEL_NAME = "gpr_dynamic_fi"
else:
    MODEL_NAME = "gpr"

outputs_for_model(MODEL_NAME)
print(f"--- Starte Tuning für: {MODEL_NAME} ---")

# --- 4) Daten laden ---
y = load_target()
X_ifo = load_ifo_features()

# Align Indizes
idx_common = y.index.intersection(X_ifo.index)
y = y.loc[idx_common]
X_ifo = X_ifo.loc[idx_common]

X_full_lagged = None
rolling_imp = None
y_fi = None

if USE_DYNAMIC_FI_PIPELINE:
    FI_BASE_DIR = PROJECT_ROOT / "outputs" / "feature_importance" / "outputs_no_missing"
    try:
        X_full_lagged = load_full_lagged_features(base_dir=FI_BASE_DIR)
        rolling_imp   = load_rolling_importance(base_dir=FI_BASE_DIR)

        idx_fi = y.index.intersection(X_full_lagged.index).intersection(rolling_imp.index)
        y_fi          = y.loc[idx_fi]
        X_full_lagged = X_full_lagged.loc[idx_fi]
        rolling_imp   = rolling_imp.loc[idx_fi]
        print(f"Dynamic FI Modus: {X_full_lagged.shape[1]} Features geladen.")
    except FileNotFoundError:
        print("FEHLER: Dynamic FI Artefakte nicht gefunden.")
        sys.exit(1)
else:
    print(f"Full FE Modus (Setup I/II): {X_ifo.shape[1]} Basis-Features.")

# --- 5) Config Defaults (Thesis Policy) ---
def get_thesis_cfg() -> GlobalConfig:
    cfg = GlobalConfig(preset="thesis")
    cfg.policy_window = 24
    cfg.policy_decay = 0.97
    cfg.selection_mode = "decayed_best"
    return cfg

cfg_obj = get_thesis_cfg()

# --- 6) Grid Definition ---------------------------------------

def build_grid_full_fe():
    """Setup I (ifo) und Setup II (ifo + TargetBlocks)."""

    # A) FE & DR (Standard Thesis Grid)
    lag_candidates = [tuple(range(7))]

    corr_opts = [
        {"corr_spec": dict(DEFAULT_CORR_SPEC)},
        {"corr_spec": dict(EWMA_CORR_SPEC)},
    ]

    k1_opts = [300, 5000, 50000]
    red_opts = [0.9, 1.0]

    dr_opts = [
        {"dr_method": "none"},
        {"dr_method": "pca", "pca_kmax": 15, "pca_var_target": 0.99},
        {"dr_method": "pca", "pca_kmax": 30, "pca_var_target": 0.99},
        {"dr_method": "pls", "pls_components": 15},
        {"dr_method": "pls", "pls_components": 30},
    ]

    # B) Setup II (Target Blocks)
    block_opts = [
        None,                                 # Setup I
        ["AR1", "Chronos"],                   # Setup II a
        ["AR1", "Chronos", "TSFresh"]         # Setup II b
    ]

    # C) Weights
    weight_opts = [
        {"sample_weight_decay": None},
        # GPR unterstützt sample_weights via alpha-Modulation (siehe src/models/GPR.py)
        {"sample_weight_decay": 0.99}
    ]

    # D) GPR Hyperparameter (Thesis Summary)
    # Kernel: ARD-RBF, ARD-Matern(1.5), ARD-Matern(2.5)
    kernel_list = ["rbf", "matern"]
    ard_list = [True] # Thesis: "ARD-..." -> immer an
    nu_list = [1.5, 2.5] # Nur für Matern

    # Optimization
    n_restarts_list = [2, 3, 5, 10, 20]

    # Noise / Init (robuste Defaults)
    noise_alpha_list = [1e-4] # Kleines Jitter für Stabilität
    length_scale_init_list = [1.0]

    grid = []

    # 1. FE Loop
    for lags, corr, k1, red, dr in product(lag_candidates, corr_opts, k1_opts, red_opts, dr_opts):

        # Plausibilitäts-Check: Bei GPR ist hohe Dimension tödlich.
        # Wenn DR=None und K1=50000, wird das extrem langsam.
        # Wir lassen es der Vollständigkeit halber drin (Thesis Constraint),
        # aber Stage A wird es vermutlich aussortieren.

        base_fe = {
            "lag_candidates": lags,
            "k1_topk": k1,
            "redundancy_param": red,
            **dr,
            **corr
        }

        # 2. Blocks & Weights
        for blocks, weights in product(block_opts, weight_opts):

            # 3. Model HPs
            for k, ard, nu, restarts, noise, ls_init in product(kernel_list, ard_list, nu_list, n_restarts_list, noise_alpha_list, length_scale_init_list):

                # Skip redundante RBF-Nu-Kombinationen
                if k == "rbf" and nu != 1.5:
                    continue # RBF ignoriert nu, wir berechnen es nur einmal (bei nu=1.5)

                hp = {
                    **base_fe,
                    "target_block_set": blocks,
                    **weights,
                    "kernel": k,
                    "ard": ard,
                    "nu": nu,
                    "n_restarts_optimizer": restarts,
                    "noise_alpha": noise,
                    "length_scale_init": ls_init,
                    "normalize_y": True, # Standard für GPR
                    "seed": SEED
                }
                grid.append(hp)

    return grid

def build_grid_dynamic_fi():
    """Setup III: Dynamic Feature Importance via strict Top-N."""

    n_features_list = [50]  # Fix 20 Features

    weight_opts = [{"sample_weight_decay": None}, {"sample_weight_decay": 0.99}]

    # GPR Params
    kernel_list = ["rbf", "matern"]
    ard_list = [True]
    nu_list = [1.5, 2.5]
    n_restarts_list = [2, 10]
    noise_alpha_list = [1e-4]
    length_scale_init_list = [1.0]

    grid = []
    for n_feat, w in product(n_features_list, weight_opts):
        for k, ard, nu, restarts, noise, ls_init in product(kernel_list, ard_list, nu_list, n_restarts_list, noise_alpha_list, length_scale_init_list):

            if k == "rbf" and nu != 1.5:
                continue

            hp = {
                "n_features_to_use": n_feat,
                **w,
                "kernel": k,
                "ard": ard,
                "nu": nu,
                "n_restarts_optimizer": restarts,
                "noise_alpha": noise,
                "length_scale_init": ls_init,
                "normalize_y": True,
                "seed": SEED
            }
            grid.append(hp)
    return grid

# --- 7) Ausführung --------------------------------------------

if USE_DYNAMIC_FI_PIPELINE:
    grid = build_grid_dynamic_fi()
    print(f"Dynamic FI Grid Größe (Setup III): {len(grid)} Konfigurationen.")

    shortlist = run_stageA(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        model_grid=grid,
        X=X_ifo, # Dummy
        y=y_fi,
        cfg=cfg_obj,
        X_full_lagged=X_full_lagged,
        rolling_imp=rolling_imp,
        keep_top_k_final=10,
        min_survivors_per_block=0
    )

    run_stageB(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        shortlist=shortlist,
        X=X_ifo, # Dummy
        y=y_fi,
        cfg=cfg_obj,
        X_full_lagged=X_full_lagged,
        rolling_imp=rolling_imp
    )

else:
    grid = build_grid_full_fe()
    print(f"Full FE Grid Größe (Setup I & II): {len(grid)} Konfigurationen.")

    shortlist = run_stageA(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        model_grid=grid,
        X=X_ifo,
        y=y,
        cfg=cfg_obj,
        keep_top_k_final=10,
        min_survivors_per_block=0
    )

    run_stageB(
        model_name=MODEL_NAME,
        model_ctor=lambda hp: ForecastModel(hp),
        shortlist=shortlist,
        X=X_ifo,
        y=y,
        cfg=cfg_obj
    )

print("\nTuning abgeschlossen.")

PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
Modell gpr wird getunt.
INFO in load_ifo_features: Renaming columns to ensure validity.
Full-FE Daten geladen. Shapes: (407, 2160) (407,)
Erstelle HP-Grid für 'Full FE'...
Optimierte HP-Kombinationen: 64
Erstes HP-Set: {'lag_candidates': (1, 2, 3, 6, 12), 'top_k_lags_per_feature': 1, 'use_rm3': True, 'k1_topk': 100, 'redundancy_param': 0.9, 'dr_method': 'none', 'corr_tag': 'expanding', 'corr_spec': {'mode': 'expanding', 'window': None, 'lam': None}, 'kernel': 'rbf', 'ard': True, 'length_scale_init': 0.5, 'nu': 1.5, 'noise_alpha': 0.0001, 'n_restarts_optimizer': 0, 'normalize_y': True, 'optimizer': 'fmin_l_bfgs_b', 'seed': 42, 'target_block_set': None, 'sample_weight_decay': None}
[Stage A] Using FULL FE pipeline (Gleis 1 & 2).
[Stage A][Block 1] train_end=180, OOS=181-200 | configs=64
  - Config 1/64
    · Month 5/20 processed | running...RMSE=1.6454
    · Month 10/20 processed | running...RMSE=1.3139
    · Month 15/20 pr

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 1 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 2 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a bette

    · Month 5/20 processed | running...RMSE=1.6454
    · Month 10/20 processed | running...RMSE=1.3139
    · Month 15/20 processed | running...RMSE=1.2336
    · Month 20/20 processed | running...RMSE=1.1010
  - Config 6/64
    · Month 5/20 processed | running...RMSE=1.6454
    · Month 10/20 processed | running...RMSE=1.3139
    · Month 15/20 processed | running...RMSE=1.2336
    · Month 20/20 processed | running...RMSE=1.1010
  - Config 7/64
    · Month 5/20 processed | running...RMSE=1.6454
    · Month 10/20 processed | running...RMSE=1.3139
    · Month 15/20 processed | running...RMSE=1.2336
    · Month 20/20 processed | running...RMSE=1.1010
  - Config 8/64
    · Month 5/20 processed | running...RMSE=1.6407
    · Month 10/20 processed | running...RMSE=1.3090
    · Month 15/20 processed | running...RMSE=1.2294
    · Month 20/20 processed | running...RMSE=1.0966
  - Config 9/64


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 1 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 2 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a bette

    · Month 5/20 processed | running...RMSE=1.6454
    · Month 10/20 processed | running...RMSE=1.3139
    · Month 15/20 processed | running...RMSE=1.2336
    · Month 20/20 processed | running...RMSE=1.1010
  - Config 10/64
    · Month 5/20 processed | running...RMSE=1.6454
    · Month 10/20 processed | running...RMSE=1.3139
    · Month 15/20 processed | running...RMSE=1.2336
    · Month 20/20 processed | running...RMSE=1.1010
  - Config 11/64
    · Month 5/20 processed | running...RMSE=1.6454
    · Month 10/20 processed | running...RMSE=1.3139
    · Month 15/20 processed | running...RMSE=1.2336
    · Month 20/20 processed | running...RMSE=1.1010
  - Config 12/64
    · Month 5/20 processed | running...RMSE=1.6407
    · Month 10/20 processed | running...RMSE=1.3090
    · Month 15/20 processed | running...RMSE=1.2294
    · Month 20/20 processed | running...RMSE=1.0966
  - Config 13/64


...

    · Month 5/20 processed | running...RMSE=0.8915
    · Month 10/20 processed | running...RMSE=1.2134
    · Month 15/20 processed | running...RMSE=2.6066
    · Month 20/20 processed | running...RMSE=2.6053
  - Config 13/32


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


    · Month 5/20 processed | running...RMSE=0.8915
    · Month 10/20 processed | running...RMSE=1.2134
    · Month 15/20 processed | running...RMSE=2.6066
    · Month 20/20 processed | running...RMSE=2.6053
  - Config 14/32
    · Month 5/20 processed | running...RMSE=0.8915
    · Month 10/20 processed | running...RMSE=1.2134
    · Month 15/20 processed | running...RMSE=2.6066
    · Month 20/20 processed | running...RMSE=2.6053
  - Config 15/32
    · Month 5/20 processed | running...RMSE=0.8917
    · Month 10/20 processed | running...RMSE=1.2129
    · Month 15/20 processed | running...RMSE=2.6029
    · Month 20/20 processed | running...RMSE=2.6025
  - Config 16/32
    · Month 5/20 processed | running...RMSE=0.8917
    · Month 10/20 processed | running...RMSE=1.2129
    · Month 15/20 processed | running...RMSE=2.6029
    · Month 20/20 processed | running...RMSE=2.6025
  - Config 17/32
    · Month 5/20 processed | running...RMSE=0.8915
    · Month 10/20 processed | running...RMSE=1.2134
 

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


[Stage B] Month origin t=290 | evaluating 5 configs | active=1
[Stage B] Month origin t=291 | evaluating 5 configs | active=1
[Stage B] Month origin t=292 | evaluating 5 configs | active=1
[Stage B] Month origin t=293 | evaluating 5 configs | active=1
[Stage B] Month origin t=294 | evaluating 5 configs | active=1
[Stage B] Month origin t=295 | evaluating 5 configs | active=1
[Stage B] Month origin t=296 | evaluating 5 configs | active=1
[Stage B] Month origin t=297 | evaluating 5 configs | active=1
[Stage B] Month origin t=298 | evaluating 5 configs | active=1
[Stage B] Month origin t=299 | evaluating 5 configs | active=1
[Stage B] Month origin t=300 | evaluating 5 configs | active=1
[Stage B] Month origin t=301 | evaluating 5 configs | active=1
[Stage B] Month origin t=302 | evaluating 5 configs | active=1
[Stage B] Month origin t=303 | evaluating 5 configs | active=1
[Stage B] Month origin t=304 | evaluating 5 configs | active=1
[Stage B] Month origin t=305 | evaluating 5 configs | a